In [1]:
from datetime import datetime, timezone

import codebook
import numpy as np
import pandas as pd
import scipy.stats as scs
import seaborn as sns
from IPython.display import display
from matplotlib import pyplot as plt

pd.options.display.max_rows = 500
pd.options.display.max_columns = 200

labels = codebook.variable_labels()
encodings = codebook.student_encodings()

In [2]:
rdf = pd.read_csv("no_weights_hsls_17_student_pets_sr_v1_0.csv")
rdf.head()

,Unnamed: 0,STU_ID,SCH_ID,X1NCESID,X2NCESID,STRAT_ID,PSU,X2UNIV1,X2UNIV2A,X2UNIV2B,X3UNIV1,X4UNIV1,X1SEX,X1RACE,X1HISPANIC,X1WHITE,X1BLACK,X1ASIAN,X1PACISLE,X1AMINDIAN,X1HISPTYPE,X1ASIANTYPE,X1NATIVELANG,X1DUALLANG,X1STDOB,X1TXMTH,X1TXMSEM,X1TXMSCR,X1TXMTSCOR,X1TXMQUINT,X1TXMPROF1,X1TXMPROF2,X1TXMPROF3,X1TXMPROF4,X1TXMPROF5,X1MACC,X1PARRESP,X1P1RELATION,X1PAR1EDU,X1PAR1EMP,X1PAR1OCC2,X1PAR1OCC6,X1PAR1OCC_STEM1,X1PAR1OCC_STEM2,X1PAR1RACE,X1P2RELATION,X1PAR2EDU,X1PAR2EMP,X1PAR2OCC2,X1PAR2OCC6,X1PAR2OCC_STEM1,X1PAR2OCC_STEM2,X1PAR2RACE,X1PAREDU,X1PARPATTERN,X1MOMRESP,X1MOMREL,X1MOMEDU,X1MOMEMP,X1MOMOCC2,X1MOMOCC6,X1MOMOCC_STEM1,X1MOMOCC_STEM2,X1MOMRACE,X1DADRESP,X1DADREL,X1DADEDU,X1DADEMP,X1DADOCC2,X1DADOCC6,X1DADOCC_STEM1,X1DADOCC_STEM2,X1DADRACE,X1HHNUMBER,X1FAMINCOME,X1POVERTY,X1POVERTY130,X1POVERTY185,X1SES,X1SESQ5,X1SES_U,X1SESQ5_U,X1MTHID,X1MTHUTI,X1MTHEFF,X1MTHINT,X1SCIID,X1SCIUTI,X1SCIEFF,X1SCIINT,X1SCHOOLBEL,X1SCHOOLENG,X1STU30OCC2,X1STU30OCC6,X1STU30OCC_STEM1,X1STU30OCC_STEM2,X1STUEDEXPCT,X1PAREDEXPCT,X1STUPRVSCHL_R,X1IEPFLAG,...,X2PAR1EMP_IM,X2PAR2EMP_IM,X2PAR1OCC_IM,X2PAR2OCC_IM,X2MOMREL_IM,X2MOMEDU_IM,X2MOMEMP_IM,X2MOMOCC_IM,X2DADREL_IM,X2DADEDU_IM,X2DADEMP_IM,X2DADOCC_IM,X2HHNUMB_IM,X2FAMINC_IM,X2POVERTY_IM,X2SES_IM,X2STUEDEX_IM,X2PAREDEX_IM,X3CLASSES_IM,X3WORK_IM,X3HSCRED_IM,X3HSCREDTY_IM,X3LASTHSD_IM,X3DROPSTAT_IM,X3EVERDROP_IM,X3CLGANDW_IM,X4X2SES1,X4X2SES2,X4X2SES3,X4X2SES4,X4X2SES5,X4X2SES1_U,X4X2SES2_U,X4X2SES3_U,X4X2SES4_U,X4X2SES5_U,X4X2SES_IM,X4HSCOMPDATE_IM,X4HSCOMPSTAT_IM,X4FB16ENRSTAT_IM,X4HS2PSMOS_IM,X4PSLFSTFB16_IM,X4ATNDCLG16FB_IM,X4EVRATNDCLG_IM,X4CHILDREN_IM,X4EMPHRSFB16_IM,X4INCOMECAT_IM,X4ANYJOB_IM,X4UNEMP16FB_IM,X4WORKING16FB_IM,X5PFYTFEDWRK_IM,X5PFYSTATEAMT_IM,X5PFYSTATNEED_IM,X5PFYSTATNOND_IM,X5PFYSTGTAMT_IM,X5PFYSTMILAMT_IM,X5PFYSTNDMRT_IM,X5PFYSTNDONLY_IM,X5PFYSTNOND1_IM,X5PFYSTVETAMT_IM,X5PFYSTWKAMT_IM,X5PFYSTMERIT_IM,X5PFYSTLNAMT_IM,X5PFYVOCHELP_IM,X5PFYINSTAMT_IM,X5PFYINGRTAMT_IM,X5PFYINSTNEED_IM,X5PFYINSMERIT_IM,X5PFYINATHAMT_IM,X5PFYINSTNOND1_IM,X5PFYEMPLWAIV_IM,X5PFYINSMILAMT_IM,X5PFYINLNAMT_IM,X5PFYINSWAIV_IM,X5PFYINSTWRK_IM,X5PFYINSTVETAMT_IM,X5PFYMERITAID_IM,X5PFYSEOGAMT_IM,X5PFYTITIVAMT_IM,X5PFYNEEDAID_IM,X5PFYTITIVAIDREC_IM,X5PFYSTAIDREC_IM,X5PFYINSTAIDREC_IM,X5PFYANYAIDREC_IM,X5PFYCAMPAMT_IM,X5PFYFEDNEED_IM,X5PFYFEDPACK_IM,X5PFYT4GRTAMT_IM,X5PFYNETPRICEALL_IM,X5PFYTOTAID2_IM,X5PFYNETPRICEGRT_IM,X5PFYPELLPACK_IM,X5PFYTOTLOAN_IM,X5PFYTOTLOAN2_IM,X5PFYTOTLOAN3_IM,X5EVRFEDAPP_IM,X5FEDAPP14_IM,X5FEDAPP15_IM,X5FEDAPP16_IM,X5PFYTUITION_IM
0,0,10001,-5,-5,-5,-5,-5,11,1,1,1111,11111,1,8,0,1,0,-5,-5,-5,-5,-5,-5,1,199502,0.8304,0.2549,50.4919,59.3710,5,0.9997,0.9669,0.8976,0.3126,0.1013,0,1,2,5,4,21,-5,0,-5,8,1,5,4,29,-5,4,-5,8,5,1,0,1,5,4,29,-5,4,-5,8,1,1,5,4,21,-5,0,-5,8,3,10,0,0,0,1.5644,5,1.6907,5,1.76,1.31,0.95,0.12,0.91,-0.33,1.10,-0.23,0.84,-1.41,25,-5,0,-5,8,6,-5,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.5184,1.5184,1.5184,1.5184,1.5184,1.8774,1.8774,1.8774,1.8774,1.8774,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,10002,-5,-5,-5,-5,-5,11,1,1,1111,11111,2,8,0,1,0,-5,-5,-5,-5,-5,-5,1,199511,-0.2956,0.2696,35.8045,47.6821,2,0.9714,0.5204,0.1768,0.0582,0.0512,-8,1,1,3,3,43,-5,0,-5,5,2,2,2,51,-5,0,-5,8,3,1,1,1,3,3,43,-5,0,-5,5,0,1,2,2,51,-5,0,-5,8,6,3,0,0,1,-0.3699,2,-0.3923,2,-0.57,1.31,0.55,-9.00,0.91,1.69,0.69,-9.00,0.05,-0.20,29,-5,4,-5,11,6,-5,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.1124,-0.1124,-0.1124,-0.1124,-0.1124,-0.0754,-0.0754,-0.0754,-0.0754,-0.0754,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8,-8
2,2,10003,-5,-5,-5,-5,-5,11,1,1,1111,11111,2,3,0,0,1,-5,-5,-5,-5,-5,-5,1,199506,1.2997,0.2555,56.0477,64.2431,5,1.0000,0.9914,0.9838,0.6606,0.2001,0,1,1,7,4,21,-5,0,-5,3,-7,-7,-7,-7,-5,-7,-5,-7,7,7,1,1,7,4,21,-5,0,-5,3,0,0,0,0,0,-

In [3]:
df = rdf.copy()

columns = [
    "STU_ID",
    "X1RACE",
    "X1WHITE",
    "X1SEX",
    "X1SESQ5",
    "X4PS1LEVEL",
    "X3REGION",
    "X1LOCALE",
    "P1MTHCOMP",
    "P1SCICOMP",
    "S1MTCHMFDIFF",
    "S1STCHMFDIFF",
    "S1MTHCOMP",
    "S1SCICOMP",
    "X1TXMQUINT",
    "S1MPERSON1",
    "S2MPERSON1",
    "S4MPERSON1",
    "S1SPERSON1",
    "S2SPERSON1",
    "S4SPERSON1",
    "S1MUNDERST",
    "S1SUNDERST",
    "X5STEM1GPA",
    "S3FIELD_STEM",
    "X4ENTMJST",
]


def factorize(df):
    return df.assign(
        **{c: df[c].cat.codes for c in df.columns if c in encodings.keys()}
    )


df = (
    df[columns]
    .pipe(
        lambda df: df.assign(
            **{
                c: codebook.to_categorical(df[c])
                for c in df.columns
                if c in encodings.keys()
            }
        )
    )
    .pipe(
        lambda df: df.assign(
            **{
                c: df[c].apply(lambda x: np.nan if x < 0 else x)
                for c in df.columns
                if c not in encodings.keys()
            }
        )
    )
    .pipe(lambda df: df.assign(INCLUDE=(df.isna() == False).apply(all, axis=1)))
)

# df.pipe(codebook.relabel).head()
df

,STU_ID,X1RACE,X1WHITE,X1SEX,X1SESQ5,X4PS1LEVEL,X3REGION,X1LOCALE,P1MTHCOMP,P1SCICOMP,S1MTCHMFDIFF,S1STCHMFDIFF,S1MTHCOMP,S1SCICOMP,X1TXMQUINT,S1MPERSON1,S2MPERSON1,S4MPERSON1,S1SPERSON1,S2SPERSON1,S4SPERSON1,S1MUNDERST,S1SUNDERST,X5STEM1GPA,S3FIELD_STEM,X4ENTMJST,INCLUDE
0,10001,"White, non-Hispanic",Student is White,Male,Fifth quintile (highest),2-year,Midwest,Rural,Males are somewhat better,Males are somewhat better,Disagree,Disagree,Females and males are the same,Females and males are the same,Fifth (highest) quintile,Strongly agree,Disagree,Agree,Agree,Agree,Strongly agree,Often,Often,3.7,No,NaN,False
1,10002,"White, non-Hispanic",Student is White,Female,Second quintile,4-year,Northeast,Rural,Males are much better,Females and males are the same,Strongly disagree,Strongly disagree,Males are somewhat better,Females and males are the same,Second quintile,Disagree,Disagree,Agree,Agree,Disagree,Agree,Sometimes,Often,NaN,No,No,False
2,10003,"Black/African-American, non-Hispanic",Student is not White,Female,Fifth quintile (highest),4-year,West,Suburb,Males are much better,Males are somewhat better,Strongly disagree,Strongly disagree,Females and males are the same,Females and males are the same,Fifth (highest) quintile,Disagree,Strongly disagree,Strongly agree,Disagree,Strongly disagree,Strongly agree,Often,Sometimes,2.7,No,Yes,True
3,10004,"White, non-Hispanic",Student is White,Female,Fifth quintile (highest),NaN,South,Suburb,Males are much better,Females are much better,Disagree,Disagree,Females and males are the same,Females and males are the same,Third (middle) quintile,Agree,NaN,NaN,Disagree,NaN,NaN,Sometimes,Sometimes,NaN,NaN,NaN,False
4,10005,"White, non-Hispanic",Student is White,Male,Fourth quintile,NaN,South,City,Females and males are the same,Females and males are the same,Strongly disagree,Agree,Females and males are the same,Females and males are the same,Fifth (highest) quintile,Strongly agree,NaN,Disagree,Disagree,Strongly agree,Disagree,Often,Often,NaN,No,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23498,35202,"Hispanic, race specified",Student is White,Female,Third quintile,NaN,South,Suburb,NaN,NaN,Strongly disagree,Strongly disagree,Females and males are the same,Females and males are the same,Fouth quintile,Agree,Agree,NaN,Strongly agree,Agree,NaN,Sometimes,Often,NaN,NaN,NaN,False
23499,35203,"Hispanic, race specified",Student is White,Female,First quintile (lowest),NaN,Northeast,Suburb,NaN,NaN,Strongly disagree,Strongly disagree,Females are much better,Females and males are the same,Second quintile,Strongly agree,Disagree,Agree,Strongly disagree,Strongly agree,Agree,Often,Never,NaN,No,NaN,False
23500,35204,"White, non-Hispanic",Student is White,Female,Third quintile,4-year,Midwest,City,Females and males are the same,Females and males are the same,Disagree,Disagree,Females are much better,Females are much better,Third (middle) quintile,Agree,Disagree,Disagree,Disagree,Agree,Agree,Often,Sometimes,0.0,No,No,True
23501,35205,"White, non-Hispanic",Student is White,Male,Fifth quintile (highest),2-year,Midwest,Suburb,Females and males are the same,Females and males are the same,Strongly disagree,Strongly disagree,Females are much better,Females are much better,Fifth (highest) quintile,Strongly disagree,Strongly disagree,Agree,Disagree,Strongly disagree,Disagree,Sometimes,Sometimes,NaN,No,No,False


In [4]:
(
    df[df["INCLUDE"]]
    .pipe(factorize)
    .to_csv("growth_analysis_data.csv", header=False, index=False)
)

In [5]:
df.pipe(factorize)

,STU_ID,X1RACE,X1WHITE,X1SEX,X1SESQ5,X4PS1LEVEL,X3REGION,X1LOCALE,P1MTHCOMP,P1SCICOMP,S1MTCHMFDIFF,S1STCHMFDIFF,S1MTHCOMP,S1SCICOMP,X1TXMQUINT,S1MPERSON1,S2MPERSON1,S4MPERSON1,S1SPERSON1,S2SPERSON1,S4SPERSON1,S1MUNDERST,S1SUNDERST,X5STEM1GPA,S3FIELD_STEM,X4ENTMJST,INCLUDE
0,10001,7,1,1,4,1,1,3,3,3,1,1,2,2,4,3,1,2,2,2,3,3,3,3.7,0,-1,False
1,10002,7,1,0,1,0,0,3,4,2,0,0,3,2,1,1,1,2,2,1,2,2,3,NaN,0,0,False
2,10003,2,0,0,4,0,3,1,4,3,0,0,2,2,4,1,0,3,1,0,3,3,2,2.7,0,1,True
3,10004,7,1,0,4,-1,2,1,4,0,1,1,2,2,2,2,-1,-1,1,-1,-1,2,2,NaN,-1,-1,False
4,10005,7,1,1,3,-1,2,0,2,2,0,2,2,2,4,3,-1,1,1,3,1,3,3,NaN,0,-1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23498,35202,4,1,0,2,-1,2,1,-1,-1,0,0,2,2,3,2,2,-1,3,2,-1,2,3,NaN,-1,-1,False
23499,35203,4,1,0,0,-1,0,1,-1,-1,0,0,0,2,1,3,1,2,0,3,2,3,0,NaN,0,-1,False
23500,35204,7,1,0,2,0,1,0,2,2,1,1,0,0,2,2,1,1,1,2,2,3,2,0.0,0,0,True
23501,35205,7,1,1,4,1,1,1,2,2,0,0,0,0,4,0,0,2,1,0,1,2,2,NaN,0,0,False


Below is the variables included in the data exploration.

In [6]:
# This is for updating the variable list in Mplus input files
print("\n".join(columns))

STU_ID
X1RACE
X1WHITE
X1SEX
X1SESQ5
X4PS1LEVEL
X3REGION
X1LOCALE
P1MTHCOMP
P1SCICOMP
S1MTCHMFDIFF
S1STCHMFDIFF
S1MTHCOMP
S1SCICOMP
X1TXMQUINT
S1MPERSON1
S2MPERSON1
S4MPERSON1
S1SPERSON1
S2SPERSON1
S4SPERSON1
S1MUNDERST
S1SUNDERST
X5STEM1GPA
S3FIELD_STEM
X4ENTMJST


Select columns of interest.

**For comparison between students who were included vs. excluded**

In [7]:
df.shape

(23503, 27)

In [8]:
df.pipe(factorize)

,STU_ID,X1RACE,X1WHITE,X1SEX,X1SESQ5,X4PS1LEVEL,X3REGION,X1LOCALE,P1MTHCOMP,P1SCICOMP,S1MTCHMFDIFF,S1STCHMFDIFF,S1MTHCOMP,S1SCICOMP,X1TXMQUINT,S1MPERSON1,S2MPERSON1,S4MPERSON1,S1SPERSON1,S2SPERSON1,S4SPERSON1,S1MUNDERST,S1SUNDERST,X5STEM1GPA,S3FIELD_STEM,X4ENTMJST,INCLUDE
0,10001,7,1,1,4,1,1,3,3,3,1,1,2,2,4,3,1,2,2,2,3,3,3,3.7,0,-1,False
1,10002,7,1,0,1,0,0,3,4,2,0,0,3,2,1,1,1,2,2,1,2,2,3,NaN,0,0,False
2,10003,2,0,0,4,0,3,1,4,3,0,0,2,2,4,1,0,3,1,0,3,3,2,2.7,0,1,True
3,10004,7,1,0,4,-1,2,1,4,0,1,1,2,2,2,2,-1,-1,1,-1,-1,2,2,NaN,-1,-1,False
4,10005,7,1,1,3,-1,2,0,2,2,0,2,2,2,4,3,-1,1,1,3,1,3,3,NaN,0,-1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23498,35202,4,1,0,2,-1,2,1,-1,-1,0,0,2,2,3,2,2,-1,3,2,-1,2,3,NaN,-1,-1,False
23499,35203,4,1,0,0,-1,0,1,-1,-1,0,0,0,2,1,3,1,2,0,3,2,3,0,NaN,0,-1,False
23500,35204,7,1,0,2,0,1,0,2,2,1,1,0,0,2,2,1,1,1,2,2,3,2,0.0,0,0,True
23501,35205,7,1,1,4,1,1,1,2,2,0,0,0,0,4,0,0,2,1,0,1,2,2,NaN,0,0,False


In [9]:
df["INCLUDE"].value_counts()

False    19901
True      3602
Name: INCLUDE, dtype: int64

# Chi-square

In [10]:
pd.crosstab(df["INCLUDE"], df["X1RACE"])

X1RACE,"Amer. Indian/Alaska Native, non-Hispanic","Asian, non-Hispanic","Black/African-American, non-Hispanic","Hispanic, no race specified","Hispanic, race specified","More than one race, non-Hispanic","Native Hawaiian/Pacific Islander, non-Hispanic","White, non-Hispanic"
INCLUDE,,,,,,,,
False,149,1614,2179,422,2982,1657,95,9797
True,16,338,271,0,393,284,15,2285


In [11]:
scs.chi2_contingency(
    pd.crosstab(
        df["INCLUDE"],
        df["X1RACE"],
    ),
    correction=False,
)

(259.14137845681614,
 3.1350939994162708e-52,
 7,
 array([[  138.58181091,  1639.46481753,  2057.72991954,   354.43348002,
          2834.62795039,  1630.22603014,    92.38787394, 10147.54811753],
        [   26.41818909,   312.53518247,   392.27008046,    67.56651998,
           540.37204961,   310.77396986,    17.61212606,  1934.45188247]]))

In [12]:
pd.crosstab(df["INCLUDE"], df["X1SEX"])

X1SEX,Female,Male
INCLUDE,,
False,9518,10377
True,2006,1596


In [13]:
scs.chi2_contingency(
    pd.crosstab(
        df["INCLUDE"],
        df["X1SEX"],
    ),
    correction=False,
)

(75.20488309750189,
 4.243163818306519e-18,
 1,
 array([[ 9757.41498915, 10137.58501085],
        [ 1766.58501085,  1835.41498915]]))

Because variable names are not self-explanatory, attach variable description to each variable name.

In [14]:
num_columns = [
    "P1MTHCOMP",
    "P1SCICOMP",
    "S1MTCHMFDIFF",
    "S1STCHMFDIFF",
    "S1MTHCOMP",
    "S1SCICOMP",
    "X1TXMQUINT",
    "S1MPERSON1",
    "S2MPERSON1",
    "S4MPERSON1",
    "S1SPERSON1",
    "S2SPERSON1",
    "S4SPERSON1",
    "S1MUNDERST",
    "S1SUNDERST",
    "X5STEM1GPA",
]

**Female descriptives**

In [15]:
df[num_columns]

,P1MTHCOMP,P1SCICOMP,S1MTCHMFDIFF,S1STCHMFDIFF,S1MTHCOMP,S1SCICOMP,X1TXMQUINT,S1MPERSON1,S2MPERSON1,S4MPERSON1,S1SPERSON1,S2SPERSON1,S4SPERSON1,S1MUNDERST,S1SUNDERST,X5STEM1GPA
0,Males are somewhat better,Males are somewhat better,Disagree,Disagree,Females and males are the same,Females and males are the same,Fifth (highest) quintile,Strongly agree,Disagree,Agree,Agree,Agree,Strongly agree,Often,Often,3.7
1,Males are much better,Females and males are the same,Strongly disagree,Strongly disagree,Males are somewhat better,Females and males are the same,Second quintile,Disagree,Disagree,Agree,Agree,Disagree,Agree,Sometimes,Often,NaN
2,Males are much better,Males are somewhat better,Strongly disagree,Strongly disagree,Females and males are the same,Females and males are the same,Fifth (highest) quintile,Disagree,Strongly disagree,Strongly agree,Disagree,Strongly disagree,Strongly agree,Often,Sometimes,2.7
3,Males are much better,Females are much better,Disagree,Disagree,Females and males are the same,Females and males are the same,Third (middle) quintile,Agree,NaN,NaN,Disagree,NaN,NaN,Sometimes,Sometimes,NaN
4,Females and males are the same,Females and males are the same,Strongly disagree,Agree,Females and males are the same,Females and males are the same,Fifth (highest) quintile,Strongly agree,NaN,Disagree,Disagree,Strongly agree,Disagree,Often,Often,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23498,NaN,NaN,Strongly disagree,Strongly disagree,Females and males are the same,Females and males are the same,Fouth quintile,Agree,Agree,NaN,Strongly agree,Agree,NaN,Sometimes,Often,NaN
23499,NaN,NaN,Strongly disagree,Strongly disagree,Females are much better,Females and males are the same,Second quintile,Strongly agree,Disagree,Agree,Strongly disagree,Strongly agree,Agree,Often,Never,NaN
23500,Females and males are the same,Females and males are the same,Disagree,Disagree,Females are much better,Females are much better,Third (middle) quintile,Agree,Disagree,Disagree,Disagree,Agree,Agree,Often,Sometimes,0.0
23501,Females and males are the same,Females and males are the same,Strongly disagree,Strongly disagree,Females are much better,Females are much better,Fifth (highest) quintile,Strongly disagree,Strongly disagree,Agree,Disagree,Strongly disagree,Disagree,Sometimes,Sometimes,NaN


In [16]:
df[df["X1SEX"] == "Female"][num_columns].pipe(factorize).describe()

,P1MTHCOMP,P1SCICOMP,S1MTCHMFDIFF,S1STCHMFDIFF,S1MTHCOMP,S1SCICOMP,X1TXMQUINT,S1MPERSON1,S2MPERSON1,S4MPERSON1,S1SPERSON1,S2SPERSON1,S4SPERSON1,S1MUNDERST,S1SUNDERST,X5STEM1GPA
count,11524.000000,11524.000000,11524.000000,11524.000000,11524.000000,11524.000000,11524.000000,11524.000000,11524.000000,11524.000000,11524.000000,11524.000000,11524.000000,11524.000000,11524.000000,6027.000000
mean,1.007723,0.967112,0.369143,0.293301,1.557792,1.638407,1.887365,1.225182,0.981170,0.710778,1.070288,1.137886,0.813259,2.126171,1.977959,2.569753
std,1.644440,1.580563,0.909307,0.963296,1.190612,1.190529,1.591814,1.128499,1.214585,1.359875,1.053176,1.202990,1.380495,1.184548,1.166604,1.042401
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,-1.000000,-1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,-1.000000,2.000000,2.000000,2.000000
50%,2.000000,2.000000,0.000000,0.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.800000
75%,2.000000,2.000000,1.000000,1.000000,2.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.400000
max,4.000000,4.000000,3.000000,3.000000,4.000000,4.000000,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000


In [17]:
np.round(df[df["X1SEX"] == "Female"][num_columns].pipe(factorize).corr(), 3)

,P1MTHCOMP,P1SCICOMP,S1MTCHMFDIFF,S1STCHMFDIFF,S1MTHCOMP,S1SCICOMP,X1TXMQUINT,S1MPERSON1,S2MPERSON1,S4MPERSON1,S1SPERSON1,S2SPERSON1,S4SPERSON1,S1MUNDERST,S1SUNDERST,X5STEM1GPA
P1MTHCOMP,1.000,0.891,0.137,0.144,0.231,0.216,0.288,0.199,0.091,0.104,0.235,0.123,0.129,0.280,0.282,0.091
P1SCICOMP,0.891,1.000,0.135,0.139,0.220,0.225,0.293,0.220,0.107,0.123,0.224,0.112,0.130,0.289,0.280,0.102
S1MTCHMFDIFF,0.137,0.135,1.000,0.480,0.305,0.297,0.264,0.250,-0.026,-0.010,0.273,-0.010,0.005,0.362,0.348,-0.007
S1STCHMFDIFF,0.144,0.139,0.480,1.000,0.281,0.274,0.262,0.246,0.002,0.019,0.246,0.007,0.022,0.347,0.328,0.007
S1MTHCOMP,0.231,0.220,0.305,0.281,1.000,0.673,0.391,0.314,-0.057,-0.020,0.424,-0.001,0.027,0.500,0.535,0.016
S1SCICOMP,0.216,0.225,0.297,0.274,0.673,1.000,0.414,0.426,0.024,0.032,0.360,-0.017,0.017,0.570,0.501,0.003
X1TXMQUINT,0.288,0.293,0.264,0.262,0.391,0.414,1.000,0.544,0.236,0.206,0.453,0.158,0.152,0.622,0.574,0.261
S1MPERSON1,0.199,0.220,0.250,0.246,0.314,0.426,0.544,1.000,0.312,0.231,0.455,0.072,0.101,0.721,0.509,0.077
S2MPERSON1,0.091,0.107,-0.026,0.002,-0.057,0.024,0.236,0.312,1.000,0.378,0.047,0.506,0.211,0.154,0.040,0.159
S4MPERSON1,0.104,0.123,-0.010,0.019,-0.020,0.032,0.206,0.231,0.378,1.000,0.068,0.187,0.750,0.136,0.066,0.197


**Male descriptives**

In [18]:
df[df["X1SEX"] == "Male"][num_columns].pipe(factorize).describe()

,P1MTHCOMP,P1SCICOMP,S1MTCHMFDIFF,S1STCHMFDIFF,S1MTHCOMP,S1SCICOMP,X1TXMQUINT,S1MPERSON1,S2MPERSON1,S4MPERSON1,S1SPERSON1,S2SPERSON1,S4SPERSON1,S1MUNDERST,S1SUNDERST,X5STEM1GPA
count,11973.000000,11973.000000,11973.000000,11973.000000,11973.00000,11973.000000,11973.000000,11973.000000,11973.000000,11973.000000,11973.000000,11973.000000,11973.00000,11973.000000,11973.000000,5151.000000
mean,1.054372,1.002005,0.407250,0.309196,1.66082,1.707258,1.862858,1.340349,1.113589,0.696818,1.177232,1.175228,0.72747,2.108077,2.006682,2.377830
std,1.730739,1.655513,1.018713,1.060064,1.37126,1.354262,1.628216,1.181424,1.277276,1.485183,1.126404,1.249603,1.47129,1.230465,1.229967,1.089181
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.00000,-1.000000,-1.000000,0.000000
25%,-1.000000,-1.000000,0.000000,-1.000000,1.00000,1.000000,0.000000,1.000000,0.000000,-1.000000,0.000000,0.000000,-1.00000,2.000000,2.000000,1.700000
50%,2.000000,2.000000,0.000000,0.000000,2.00000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.00000,3.000000,2.000000,2.600000
75%,2.000000,2.000000,1.000000,1.000000,2.00000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.00000,3.000000,3.000000,3.200000
max,4.000000,4.000000,3.000000,3.000000,4.00000,4.000000,4.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.00000,3.000000,3.000000,4.000000


In [19]:
np.round(df[df["X1SEX"] == "Male"][num_columns].pipe(factorize).corr(), 3)

,P1MTHCOMP,P1SCICOMP,S1MTCHMFDIFF,S1STCHMFDIFF,S1MTHCOMP,S1SCICOMP,X1TXMQUINT,S1MPERSON1,S2MPERSON1,S4MPERSON1,S1SPERSON1,S2SPERSON1,S4SPERSON1,S1MUNDERST,S1SUNDERST,X5STEM1GPA
P1MTHCOMP,1.000,0.900,0.125,0.122,0.225,0.214,0.317,0.267,0.164,0.181,0.246,0.158,0.176,0.299,0.276,0.099
P1SCICOMP,0.900,1.000,0.130,0.124,0.210,0.212,0.299,0.244,0.145,0.165,0.245,0.153,0.167,0.290,0.276,0.087
S1MTCHMFDIFF,0.125,0.130,1.000,0.468,0.283,0.302,0.246,0.262,-0.025,0.021,0.264,-0.014,0.025,0.348,0.336,-0.010
S1STCHMFDIFF,0.122,0.124,0.468,1.000,0.270,0.258,0.254,0.258,0.004,0.039,0.251,-0.007,0.030,0.329,0.320,0.029
S1MTHCOMP,0.225,0.210,0.283,0.270,1.000,0.708,0.459,0.527,0.105,0.108,0.425,0.042,0.075,0.583,0.514,0.037
S1SCICOMP,0.214,0.212,0.302,0.258,0.708,1.000,0.417,0.434,0.028,0.068,0.482,0.051,0.079,0.543,0.566,0.028
X1TXMQUINT,0.317,0.299,0.246,0.254,0.459,0.417,1.000,0.573,0.256,0.235,0.493,0.194,0.190,0.634,0.589,0.251
S1MPERSON1,0.267,0.244,0.262,0.258,0.527,0.434,0.573,1.000,0.275,0.197,0.522,0.093,0.097,0.723,0.550,0.093
S2MPERSON1,0.164,0.145,-0.025,0.004,0.105,0.028,0.256,0.275,1.000,0.351,0.084,0.613,0.239,0.145,0.063,0.157
S4MPERSON1,0.181,0.165,0.021,0.039,0.108,0.068,0.235,0.197,0.351,1.000,0.087,0.237,0.828,0.137,0.095,0.213


In [20]:
col_na = round(100 * (df.isna().sum() / len(df)), 2)
col_nunique = df.nunique()
df_summary = pd.DataFrame(
    {
        "data_type": df.dtypes,
        "percent_missing_values": col_na,
        "total_unique_values": col_nunique,
    }
)
df_summary.pipe(codebook.relabel, axis=0)

,data_type,percent_missing_values,total_unique_values
Student ID,int64,0.00,23503
X1 Student's race/ethnicity-composite,category,4.28,8
X1 Student is White-composite,category,2.40,2
X1 Student's sex,category,0.03,2
X1 Quintile coding of X1SES composite,category,8.76,5
X4 First postsecondary institution level,category,44.87,3
X3 School region,category,5.55,4
X1 School locale (urbanicity),category,0.00,4
P1 E05A Comparison of females' and males' abilities in math,category,37.17,5
P1 E05B Comparison of females' and males' abilities in science,category,37.28,5


## 1. Demographics

### 1.1 Frequencies and descriptives

In [21]:
(
    df["X1RACE"]
    .value_counts(sort=False, dropna=False, normalize=True)
    .pipe(codebook.relabel)
    .to_frame()
)

,X1 Student's race/ethnicity-composite
"Amer. Indian/Alaska Native, non-Hispanic",0.007020
"Asian, non-Hispanic",0.083053
"Black/African-American, non-Hispanic",0.104242
"Hispanic, no race specified",0.017955
"Hispanic, race specified",0.143599
"More than one race, non-Hispanic",0.082585
"Native Hawaiian/Pacific Islander, non-Hispanic",0.004680
"White, non-Hispanic",0.514062
NaN,0.042803


In [22]:
(
    df["X1WHITE"]
    .value_counts(dropna=False, normalize=True)
    .pipe(codebook.relabel)
    .to_frame()
)

,X1 Student is White-composite
Student is White,0.723567
Student is not White,0.252393
NaN,0.024039


Note: Not sure how the first category's count is 17,006, in relation to the frequencies for `X1RACE` (see above).

In [23]:
(
    df["X1SEX"]
    .value_counts(dropna=False, normalize=True)
    .pipe(codebook.relabel)
    .to_frame()
)

,X1 Student's sex
Male,0.509424
Female,0.490320
NaN,0.000255


In [24]:
(
    df["X1SESQ5"]
    .value_counts(dropna=False, normalize=True)
    .pipe(codebook.relabel)
    .to_frame()
)

,X1 Quintile coding of X1SES composite
Fifth quintile (highest),0.234821
Fourth quintile,0.193720
Third quintile,0.180105
Second quintile,0.157639
First quintile (lowest),0.146109
NaN,0.087606


In [25]:
(
    df["X3REGION"]
    .value_counts(dropna=False, normalize=True)
    .pipe(codebook.relabel)
    .to_frame()
)

,X3 School region
South,0.384504
Midwest,0.251670
West,0.159469
Northeast,0.148875
NaN,0.055482


In [26]:
(
    df["X1LOCALE"]
    .value_counts(dropna=False, normalize=True)
    .pipe(codebook.relabel)
    .to_frame()
)

,X1 School locale (urbanicity)
Suburb,0.360252
City,0.284602
Rural,0.236523
Town,0.118623


In [27]:
df.describe()

,STU_ID,X5STEM1GPA
count,23503.000000,11179.000000
mean,22594.785304,2.481116
std,7269.600664,1.068609
min,10001.000000,0.000000
25%,16300.500000,1.900000
50%,22583.000000,2.700000
75%,28874.500000,3.300000
max,35206.000000,4.000000
